In [1]:
!pip install gql[all]

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import asyncio
from gql import gql, Client
from gql.transport.aiohttp import AIOHTTPTransport
import json
import numpy as np

### Note
You are limited to quering for 1000 values. To get more than 1000, you need to skip in batches of 1000. There is also a value limit of 5000 for setting the skip parameter. There is something like ~25,000 swaps in a given day for the ETH/USDC (0.05% fee) pool. I only pulled the first 5000. To get the rest I woud take the last timestamp as the timestamp_gte parameter and repeat. 

Also I had a data limit issue. Increasing the data rate limit fixes it 
Run this in terminal <br>
`jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10 `

In [3]:
skip_count = np.arange(0,5001,1000)
swaps_query_text = """{
  pool(id: "0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640") {
    swaps(
      where: {timestamp_gte: "1681448400", timestamp_lte: "1681534799"}
      skip: 0
      first: 1000
      orderBy: timestamp
      orderDirection: asc
    ) {
      amount0
      amount1
      amountUSD
      tick
      timestamp
    }
  }
}"""


In [4]:
### setting up the queries with skip values of 0 to 5000
first_half_query = """{
  pool(id: "0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640") {
    swaps(
      where: {timestamp_gte: "1681448400", timestamp_lte: "1681534799"}"""
skip_val = """ 
      skip: 0"""
second_half_query = """ 
      first: 1000
      orderBy: timestamp
      orderDirection: asc
    ) {
      amount0
      amount1
      amountUSD
      tick
      timestamp
    }
  }
}"""


In [5]:
v3_query_list=[swaps_query_text]

In [6]:
for i in range(len(skip_count)-1):
    skip_val = skip_val.replace(str(skip_count[i]),str(skip_count[i+1]))
    query = first_half_query+skip_val+second_half_query
    v3_query_list.append(query)

In [7]:
v3_query_list


['{\n  pool(id: "0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640") {\n    swaps(\n      where: {timestamp_gte: "1681448400", timestamp_lte: "1681534799"}\n      skip: 0\n      first: 1000\n      orderBy: timestamp\n      orderDirection: asc\n    ) {\n      amount0\n      amount1\n      amountUSD\n      tick\n      timestamp\n    }\n  }\n}',
 '{\n  pool(id: "0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640") {\n    swaps(\n      where: {timestamp_gte: "1681448400", timestamp_lte: "1681534799"} \n      skip: 1000 \n      first: 1000\n      orderBy: timestamp\n      orderDirection: asc\n    ) {\n      amount0\n      amount1\n      amountUSD\n      tick\n      timestamp\n    }\n  }\n}',
 '{\n  pool(id: "0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640") {\n    swaps(\n      where: {timestamp_gte: "1681448400", timestamp_lte: "1681534799"} \n      skip: 2000 \n      first: 1000\n      orderBy: timestamp\n      orderDirection: asc\n    ) {\n      amount0\n      amount1\n      amountUSD\n      tick\n      tim

In [8]:
day_data = []
### this is the subgraph I am quering on
uniswap_v3 = "https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3"

In [9]:
for i in range(len(v3_query_list)):
    async def main():

        transport = AIOHTTPTransport(url=uniswap_v3)
        async with Client(
            transport=transport,
            fetch_schema_from_transport=True,
        ) as session: 
        
            query = gql(v3_query_list[i])

            result = await session.execute(query)
            day_data.append(result)
        
    await main()


In [10]:
day_data

[{'pool': {'swaps': [{'amount0': '6453.06796',
     'amount1': '-3.048536092546408988',
     'amountUSD': '6447.43103310898934500711662133578',
     'tick': '199748',
     'timestamp': '1681448423'},
    {'amount0': '-785.98995',
     'amount1': '0.371688623040263322',
     'amountUSD': '785.6984740832397900399905975068295',
     'tick': '199748',
     'timestamp': '1681448423'},
    {'amount0': '-537.550081',
     'amount1': '0.2542',
     'amountUSD': '537.347235824218857911020030364751',
     'tick': '199748',
     'timestamp': '1681448435'},
    {'amount0': '10000',
     'amount1': '-4.724094977742180402',
     'amountUSD': '9991.19024033176322973201302396888',
     'tick': '199748',
     'timestamp': '1681448483'},
    {'amount0': '-1057.335004',
     'amount1': '0.5',
     'amountUSD': '1056.936965658967069061801790646639',
     'tick': '199748',
     'timestamp': '1681448483'},
    {'amount0': '-2135.857102',
     'amount1': '1.01',
     'amountUSD': '2135.0328675911134795048396

In [11]:
#with open(r"C:\Users\steph\Projects\Uniswap\swaps_day_data.json","w") as outfile:
    #json.dump(day_data, outfile)

### General issues with liquidity values at ticks
I could not find a 1 to 1 relationship between swaps and the corresponding tick data at the swap. The hourly pool data has liquidity values, but they are for the entire pool, not at specific ticks (24 values for a 1 day timeframe).
Running the following only returns 1 value with no liquidity information. <br>
`{
  pool(id: "0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640") {
    ticks(
      where: {createdAtTimestamp_gte: "1681448400", createdAtTimestamp_lte: "1681534799"}
      first: 1000
      orderBy: createdAtTimestamp
      orderDirection: asc
    ) {
      liquidityNet
      liquidityGross
      createdAtTimestamp
      price1
      price0
      volumeToken0
      volumeToken1
    }
  }
}`
You get the same result going through ticks first with a specified pool <br>
`{
ticks(
    where: {poolAddress: "0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640", createdAtTimestamp_gte: "1681448400", createdAtTimestamp_lte: "1681534799"}
    first: 1000
  ) {
    price1
    price0
    liquidityNet
    liquidityGross
    feesUSD
    createdAtTimestamp
  }
}`

In [12]:
hour_tick_query = """{
  pool(id: "0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640") {
    poolHourData(
      orderBy: periodStartUnix
      orderDirection: desc
      first: 1000
      where: {periodStartUnix_gte: 1681448400, periodStartUnix_lte: 1681534799}
    ) {
      liquidity
      tick
      periodStartUnix
      low
      high
      token0Price
      token1Price
      tvlUSD
    }
  }
}"""

In [13]:
tick_data_hourly=[]

In [14]:

async def main():

    transport = AIOHTTPTransport(url=uniswap_v3)
    async with Client(
        transport=transport,
        fetch_schema_from_transport=True,
    ) as session: 
        
        query = gql(hour_tick_query)

        result = await session.execute(query)
        tick_data_hourly.append(result)
        
await main()


In [20]:
#with open(r"C:\Users\steph\Projects\Uniswap\tick_hourly_data.json","w") as outfile:
    #json.dump(tick_data_hourly, outfile)
    
tick_data_hourly

[{'pool': {'poolHourData': [{'liquidity': '22356924693027331110',
     'tick': '199882',
     'periodStartUnix': 1681531200,
     'low': '2085.158819374747476947701443922727',
     'high': '2090.491408643884770507698028390939',
     'token0Price': '2087.664616407395553672288690627411',
     'token1Price': '0.0004790041427827006107765164411268521',
     'tvlUSD': '478182811.4486370048163724797685778'},
    {'liquidity': '25739848773689859804',
     'tick': '199868',
     'periodStartUnix': 1681527600,
     'low': '2079.527104874586122674179246815216',
     'high': '2094.383611133897799834147919308514',
     'token0Price': '2090.443671310036278642684542562293',
     'token1Price': '0.0004783673503019200783278274810573675',
     'tvlUSD': '470500982.5977946246661959862525754'},
    {'liquidity': '21406898031586430504',
     'tick': '199863',
     'periodStartUnix': 1681524000,
     'low': '2084.531526202887422267502822727144',
     'high': '2092.090107106607436458929027296863',
     'toke

###  Issues with liquidity values at ticks continued
I tried expanding further down with a specified pool and just got the same result from before, just repeated 1000 times still with no liquidity info. 

The next step is to try to generate the liquidity changes with the swap info.

In [16]:
no_liquidity_tick_query = """{
  pool(id: "0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640") {
    poolHourData {
      pool {
        ticks(where: {createdAtTimestamp_gte: "1681448400", createdAtTimestamp_lte: "1681534799"},orderBy: createdAtTimestamp, orderDirection: desc, first: 1000) {
          liquidityGross
          liquidityNet
          price0
          price1
          createdAtTimestamp
          tickIdx
        }
      }
    }
  }
}"""

In [17]:
no_liquidity_tick=[]

`
async def main():

    transport = AIOHTTPTransport(url=uniswap_v3)
    async with Client(
        transport=transport,
        fetch_schema_from_transport=True,
    ) as session: 
        
        query = gql(no_liquidity_tick_query)

        result = await session.execute(query)
        no_liquidity_tick.append(result)
        
await main()`


In [22]:
#no_liquidity_tick
#with open(r"C:\Users\steph\Projects\Uniswap\tick_no_liquidity_data.json","w") as outfile:
    #json.dump(no_liquidity_tick, outfile)